# Benchmark of various outlier detection models

**[PyOD](https://github.com/yzhao062/Pyod)** is a comprehensive **Python toolkit** to **identify outlying objects** in 
multivariate data with both unsupervised and supervised approaches.

The following models are used for comparison:

  1. Linear Models for Outlier Detection:
     1. **PCA: Principal Component Analysis** use the sum of
       weighted projected distances to the eigenvector hyperplane 
       as the outlier outlier scores) [10]
     2. **MCD: Minimum Covariance Determinant** (use the mahalanobis distances 
       as the outlier scores) [11, 12]
     3. **One-Class Support Vector Machines** [3]
     
  2. Proximity-Based Outlier Detection Models:
     1. **LOF: Local Outlier Factor** [1]
     2. **kNN: k Nearest Neighbors** (use the distance to the kth nearest 
     neighbor as the outlier score)
     3. **Average kNN** Outlier Detection (use the average distance to k 
     nearest neighbors as the outlier score)
     4. **Median kNN** Outlier Detection (use the median distance to k nearest 
     neighbors as the outlier score)
     5. **HBOS: Histogram-based Outlier Score** [5]
     
  3. Probabilistic Models for Outlier Detection:
     1. **ABOD: Angle-Based Outlier Detection** [7]
     2. **FastABOD: Fast Angle-Based Outlier Detection using approximation** [7]
  
  4. Outlier Ensembles and Combination Frameworks
     1. **Isolation Forest** [2]
     2. **Feature Bagging** [9]

In [45]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [46]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD', 'OCSVM',
              'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Feature Bagging': FeatureBagging(
            contamination=outliers_fraction,
            random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(
            contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction),
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)
        
        


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.2267 s
Feature Bagging ROC:0.7806, precision @ rank n:0.4643, execution time: 0.5686 s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.2096 s
Isolation Forest ROC:0.8499, precision @ rank n:0.5714, execution time: 0.2507 s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.0852 s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0712 s
Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 0.5064 s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.0461 s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0552 s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6618, precision @ rank n:0.2571, execution time: 0.5657 s
Feature Bagg

Minimum Covariance Determinant (MCD) ROC:0.812, precision @ rank n:0.0526, execution time: 1.8573 s
One-class SVM (OCSVM) ROC:0.9175, precision @ rank n:0.2456, execution time: 0.9939 s
Principal Component Analysis (PCA) ROC:0.9231, precision @ rank n:0.2105, execution time: 0.007 s

... Processing pima.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7186, precision @ rank n:0.534, execution time: 0.2318 s
Feature Bagging ROC:0.6497, precision @ rank n:0.4369, execution time: 0.1003 s
Histogram-base Outlier Detection (HBOS) ROC:0.7533, precision @ rank n:0.5825, execution time: 0.0101 s
Isolation Forest ROC:0.6941, precision @ rank n:0.5146, execution time: 0.2708 s
K Nearest Neighbors (KNN) ROC:0.7303, precision @ rank n:0.5437, execution time: 0.0341 s
Local Outlier Factor (LOF) ROC:0.6451, precision @ rank n:0.4272, execution time: 0.0291 s
Minimum Covariance Determinant (MCD) ROC:0.6987, precision @ rank n:0.5049, execution time: 0.0552 s
One-class SVM (OCSVM) ROC:0.6429, precisi

Analyze the performance of ROC and Precision @ n

In [52]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.2267,0.5686,0.2096,0.2507,0.0852,0.0712,0.5064,0.0461,0.0552
0,cardio,1831,21,9.6122,0.5657,0.7983,0.0542,0.2547,0.1304,0.0993,0.4633,0.0842,0.004
0,glass,214,9,4.2056,0.0742,0.0361,0.007,0.1885,0.0081,0.003,0.0431,0.002,0.001
0,ionosphere,351,33,35.8974,0.1554,0.0702,0.0271,0.1725,0.016,0.007,0.0592,0.006,0.002
0,letter,1600,32,6.25,0.5596,0.7542,0.0752,0.2818,0.2176,0.1113,1.3248,0.0862,0.005
0,lympho,148,18,4.0541,0.0522,0.0261,0.01,0.1765,0.005,0.002,0.0411,0.002,0.002
0,mnist,7603,100,9.2069,8.3991,50.7403,1.1533,1.9235,6.279,6.6791,2.5353,4.88,0.1464
0,musk,3062,166,3.1679,2.455,14.6439,0.6609,1.053,1.8964,1.8353,9.2003,1.2466,0.1444
0,optdigits,5216,64,2.8758,2.9244,12.4657,0.5355,0.7381,1.8463,1.6317,0.9607,1.4321,0.0461
0,pendigits,6870,16,2.2707,2.3146,4.2753,0.1535,0.5857,0.5987,0.6077,1.8573,0.9939,0.007


In [53]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7806,0.8511,0.8499,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.6618,0.6281,0.8755,0.9437,0.7665,0.6044,0.8466,0.9327,0.9488
0,glass,214,9,4.2056,0.755,0.8514,0.7068,0.6827,0.8394,0.8153,0.7912,0.5703,0.6265
0,ionosphere,351,33,35.8974,0.9275,0.9228,0.6025,0.8655,0.9245,0.9279,0.9459,0.8353,0.7926
0,letter,1600,32,6.25,0.8871,0.8956,0.6115,0.6446,0.8859,0.8832,0.8294,0.5922,0.5262
0,lympho,148,18,4.0541,0.8362,0.9741,1,0.9914,0.9741,0.9741,0.9483,0.9741,0.9828
0,mnist,7603,100,9.2069,0.7796,0.7209,0.5682,0.8156,0.8443,0.7035,0.8769,0.8569,0.8558
0,musk,3062,166,3.1679,0.1806,0.7416,1,1,0.7959,0.7692,1,1,1
0,optdigits,5216,64,2.8758,0.4699,0.4581,0.8578,0.7175,0.363,0.4692,0.3927,0.4732,0.4931
0,pendigits,6870,16,2.2707,0.7067,0.5604,0.9246,0.9398,0.7705,0.5514,0.812,0.9175,0.9231


In [55]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.2571,0.1,0.4286,0.5429,0.3571,0.0714,0.5,0.5,0.5714
0,glass,214,9,4.2056,0.3333,0.3333,0,0,0.3333,0.3333,0,0.3333,0.3333
0,ionosphere,351,33,35.8974,0.8364,0.7818,0.3818,0.7091,0.8545,0.7818,0.8545,0.7091,0.6364
0,letter,1600,32,6.25,0.4091,0.5227,0.1136,0.1591,0.4318,0.4773,0.3182,0.2273,0.1591
0,lympho,148,18,4.0541,0.5,0.5,1,0.5,0.5,0.5,0.5,0.5,0.5
0,mnist,7603,100,9.2069,0.3563,0.3065,0.1111,0.318,0.4291,0.3027,0.3487,0.4176,0.3985
0,musk,3062,166,3.1679,0.1081,0.2973,1,1,0.2973,0.2973,1,1,0.973
0,optdigits,5216,64,2.8758,0,0.0172,0.2241,0.0172,0,0.0172,0,0,0
0,pendigits,6870,16,2.2707,0.0526,0.0702,0.2105,0.1228,0.0702,0.0702,0.0526,0.2456,0.2105
